In [1]:
import torch
from pathlib import Path
from functools import partial
from pytorch_accelerated.callbacks import (
    EarlyStoppingCallback,
    SaveBestModelCallback,
    get_default_callbacks,
)
from pytorch_accelerated import Trainer
from yolov7 import create_yolov7_model
from yolov7.dataset import create_yolov7_transforms
from yolov7.loss_factory import create_yolov7_loss
from yolov7.trainer import (
    Yolov7Trainer,
    filter_eval_predictions,
)

from PIL import Image, ImageDraw, ImageFont
import numpy as np

# === 1) Helper: run_inference using YOLOv7’s non_max_suppression ===
def run_inference(
    model,
    image_path: str,
    transforms,
    filter_fn,
    device: torch.device = None,
    conf_threshold: float = 0.25,
):
    # 1) Prep
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()

    # 2) Load + Transform
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)
    tr_out = transforms(image=img_np, bboxes=[], labels=[])
    img_t = tr_out["image"]
    if isinstance(img_t, np.ndarray):
        x = torch.from_numpy(img_t).permute(2,0,1).float().unsqueeze(0)
    else:
        x = img_t.unsqueeze(0)
    x = x.to(device)

    # 3) Forward + model.postprocess → list of per-image detections
    with torch.no_grad():
        fpn_out = model(x)
        # this returns List[Tensor] with shape [N,6]
        preds = model.postprocess(fpn_out, conf_thres=conf_threshold)

    # 4) Run the exact NMS callback you used in training
    if filter_fn is not None:
        preds = filter_fn(preds)  

    # 5) Extract the first (and only) image’s detections
    det = preds[0]  
    if det is None or det.shape[0] == 0:
        return img, torch.empty((0,4)), torch.empty((0,)), torch.empty((0,),dtype=torch.long)

    boxes = det[:, :4].cpu()
    scores = det[:, 4].cpu()
    class_ids = det[:, 5].long().cpu()

    return img, boxes, scores, class_ids

# === 2) Rebuild trainer exactly as in training ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 1
model = create_yolov7_model("yolov7", num_classes=num_classes, pretrained=False)
loss_func = create_yolov7_loss(model, image_size=416)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

callbacks = [
    # you only need SaveBestModelCallback if that's what produced best_model.ckpt
    SaveBestModelCallback(watch_metric="map", greater_is_better=True),
    EarlyStoppingCallback(early_stopping_patience=3, watch_metric="map", greater_is_better=True),
    *get_default_callbacks(progress_bar=False),
]
trainer = Yolov7Trainer(
    model=model,
    loss_func=loss_func,
    optimizer=optimizer,
    callbacks=callbacks,
    filter_eval_predictions_fn=partial(filter_eval_predictions, confidence_threshold=0.01, nms_threshold=0.3),
)

# === 3) Load the checkpoint Chris’s callback saved ===
ckpt = Path("/home/satyam/Dev/Yolov7-training/cars_model.pt")  # or .pt
trainer.load_checkpoint(ckpt, load_scheduler=False)

# now trainer.get_model() is your fine-tuned YOLOv7
ft_model = trainer.get_model()

# === 4) Inference on a sample image ===
infer_transforms = create_yolov7_transforms(training=False, image_size=(416,416))
sample_image = "/home/satyam/Dev/data/cars/testing_images/vid_5_400.jpg"
pil_img, boxes, scores, class_ids = run_inference(
    ft_model,
    sample_image,
    transforms=infer_transforms,
    filter_fn=partial(filter_eval_predictions, confidence_threshold=0.25, nms_threshold=0.3),
    device=device,
    conf_threshold=0.25,
)

# === 5) Draw & display ===
draw = ImageDraw.Draw(pil_img)
font = ImageFont.load_default()
for (x1,y1,x2,y2), s in zip(boxes, scores):
    draw.rectangle([x1,y1,x2,y2], outline="red", width=2)
    label = f"car {s:.2f}"
    tw, th = font.getbbox(label)[2:]
    draw.rectangle([x1, y1-th, x1+tw, y1], fill="red")
    draw.text((x1, y1-th), label, fill="white", font=font)

pil_img.show()


/home/satyam/anaconda3/envs/yolo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
